In [2]:
from pyxll_notebook.server import xl_func

In [3]:
@xl_func
def test_func(a, b):
    """test func takes two args
    :param a: a
    :param b: b
    """
    return a + b

In [4]:
@xl_func("int, int: int[][]", auto_resize=True)
def auto_resize_test(rows, cols):
    result = []
    i = 0
    for r in range(rows):
        row = []
        for c in range(cols):
            row.append(i)
            i += 1
        result.append(row)
    return result